In [14]:
import json
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import streamlit as st
from langchain.retrievers import WikipediaRetriever
from langchain.schema import BaseOutputParser, output_parser



function = {
    "name": "create_quiz",
    "description": "이 function은 주어진 제시어를 이용해 문제와 답안이 있는 퀴즈를 만들어 준다. 질문과 답변은 한국어로 만들어줘.",
    # "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type":"string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            },
        },
        "required": ["questions"],
    },
}



llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ]
)


prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})

response = response.additional_kwargs["function_call"]["arguments"]

response


'{"questions":[{"question":"어떤 제국이 로마 제국을 정복했나요?","answers":[{"answer":"카르타고","correct":false},{"answer":"프랑스","correct":false},{"answer":"그리스","correct":false},{"answer":"스페인","correct":true}]},{"question":"로마 제국의 수도는 어디에 있었나요?","answers":[{"answer":"파리","correct":false},{"answer":"로마","correct":true},{"answer":"아테네","correct":false},{"answer":"마드리드","correct":false}]},{"question":"로마 제국의 창시자는 누구인가요?","answers":[{"answer":"율리우스 시저","correct":false},{"answer":"아우구스투스","correct":true},{"answer":"네로","correct":false},{"answer":"카이사르","correct":false}]}]}'

In [15]:
import json 

json.loads(response)
# response
# response["questions"]

{'questions': [{'question': '어떤 제국이 로마 제국을 정복했나요?',
   'answers': [{'answer': '카르타고', 'correct': False},
    {'answer': '프랑스', 'correct': False},
    {'answer': '그리스', 'correct': False},
    {'answer': '스페인', 'correct': True}]},
  {'question': '로마 제국의 수도는 어디에 있었나요?',
   'answers': [{'answer': '파리', 'correct': False},
    {'answer': '로마', 'correct': True},
    {'answer': '아테네', 'correct': False},
    {'answer': '마드리드', 'correct': False}]},
  {'question': '로마 제국의 창시자는 누구인가요?',
   'answers': [{'answer': '율리우스 시저', 'correct': False},
    {'answer': '아우구스투스', 'correct': True},
    {'answer': '네로', 'correct': False},
    {'answer': '카이사르', 'correct': False}]}]}